## Install packages and import libraries

In [ ]:
# Install Vertex AI SDK & Other dependencies

!sudo apt -y -qq install tesseract-ocr
!sudo apt -y -qq install libtesseract-dev
!sudo apt-get -y -qq install poppler-utils #required by PyPDF2 for page count and other pdf utilities
!sudo apt-get -y -qq install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig
!pip install langchain langchain-community langchain-core langchain-google-vertexai pypdf

In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()

app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# Import libraries

import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain_google_vertexai import ChatVertexAI

warnings.filterwarnings("ignore")

## 1. Load the model and prepare data

In [2]:
#Load the pre-trained text generation model named "gemini-1.5-pro" using "ChatVertexAI" class.
llm = ChatVertexAI(
    model="gemini-2.5-pro",
    temperature=0,
    max_tokens=None,
    max_retries=6,
    stop=None,
    # other params...
)

In [4]:
#Download a PDF file from specified URL and save it in "data" directory.
from pathlib import Path
import urllib.request

url = 'https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf'

data_dir = Path('data')
data_dir.mkdir(parents=True, exist_ok=True)

pdf_path = data_dir/"practitioners_guaide_to_mlops_whitepaper.pdf"

urllib.request.urlretrieve(url, pdf_path)
print(f"PDF downloaded to: {pdf_path.resolve()}")

PDF downloaded to: /home/jupyter/data/practitioners_guaide_to_mlops_whitepaper.pdf


In [15]:
#Load the PDF file and split it into individual pages.
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(str(pdf_path))
pages = loader.load_and_split()

In [16]:
print(f"{pages[2].metadata}\n")
print(pages[2].page_content)

{'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 16.1 (Macintosh)', 'creationdate': '2021-05-11T22:05:38+03:00', 'moddate': '2021-05-11T22:05:50+03:00', 'trapped': '/False', 'source': 'data/practitioners_guaide_to_mlops_whitepaper.pdf', 'total_pages': 37, 'page': 2, 'page_label': '3'}

Executive summary
Across industries, DevOps and DataOps have been widely adopted as methodologies to improve quality and re-
duce the time to market of software engineering and data engineering initiatives. With the rapid growth in machine 
learning (ML) systems, similar approaches need to be developed in the context of ML engineering, which handle the 
unique complexities of the practical applications of ML. This is the domain of MLOps. MLOps is a set of standard-
ized processes and technology capabilities for building, deploying, and operationalizing ML systems rapidly and 
reliably.]
We previously published Google Cloud’s AI Adoption Framework to provide guidance for technology leader

## 2. Generate summaries

In [9]:
#Prompt design with Stuffing Chain

prompt_template = """Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
  """

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

In [10]:
#Set up a summarization chain using the stuff method. Incorporate the model loaded earlier into the summarization chain to enhance the quality of the summary.
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Summarize this content: {context}")
chain = create_stuff_documents_chain(llm, prompt)

result = chain.invoke({"context": pages})
result

'Of course. Here is a summary of the white paper "Practitioners guide to MLOps: A framework for continuous delivery and automation of machine learning."\n\n### Executive Summary\n\nThis white paper presents a comprehensive framework for MLOps (Machine Learning Operations), a methodology designed to build, deploy, and operationalize Machine Learning (ML) systems efficiently and reliably. The authors argue that many ML projects fail to move from pilot to production because they lack standardized, automated processes. MLOps addresses this by applying principles from DevOps and DataOps to the unique challenges of the ML lifecycle, such as managing data, tracking experiments, and monitoring models for performance degradation.\n\nThe framework is intended to help organizations improve collaboration, increase the reliability and scalability of their ML systems, and shorten development cycles, ultimately driving more business value from their AI/ML investments.\n\n### The Core Problem MLOps So

## 3. Add tools to the LLM for multiplication

In [61]:
#Create a tool called 'multiply' that takes two integers, 'a' and 'b', as inputs, performs the multiplication (a * b), and returns the result.
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply.invoke({"a": 3, "b": 12})

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


36

In [65]:
#Add the tools to the LLM created earlier and invoke it with the following query. Print the result in the console.
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
query = "What is 3 * 12?"
messages = [HumanMessage(query)]

llm_with_tools = llm.bind_tools([multiply])
ai_msg = llm_with_tools.invoke(messages)

In [66]:
#Iterate through the tools in the response, invoke the tools and append the response to the messages object.
llm_with_tools.invoke(messages).tool_calls
messages.append(ai_msg)

In [67]:
#Invoke the LLM with the solution of the tool and the original message and print the final user response.
for tool_call in ai_msg.tool_calls:
    selected_tool = {"multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
                    
messages

[HumanMessage(content='What is 3 * 12?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 12.0}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 21, 'candidates_token_count': 5, 'total_token_count': 252, 'prompt_tokens_details': [{'modality': 1, 'token_count': 21}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 226, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -2.4901525497436525, 'model_name': 'gemini-2.5-pro'}, id='run--8a46a23b-1199-44be-9b1a-44e9b9be9de1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '129ff52d-8c73-4baf-930a-e19e95aae92c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 21, 'output_tokens': 5, 'total_tokens': 252, 'input_token_details': {'cache_read': 0}, 'output_token_d

In [ ]:
# Additional query
query2 = "how about 13*14?"
messages.append(HumanMessage(query2))

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
                    
messages

[HumanMessage(content='What is 3 * 12?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 12.0}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 21, 'candidates_token_count': 5, 'total_token_count': 252, 'prompt_tokens_details': [{'modality': 1, 'token_count': 21}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 226, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -2.4901525497436525, 'model_name': 'gemini-2.5-pro'}, id='run--8a46a23b-1199-44be-9b1a-44e9b9be9de1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '129ff52d-8c73-4baf-930a-e19e95aae92c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 21, 'output_tokens': 5, 'total_tokens': 252, 'input_token_details': {'cache_read': 0}, 'output_token_d